In [1]:
%matplotlib inline
import pandas as pd
import random
import numpy as np
from random import sample

In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [6]:
df_inmates = pd.read_csv('exinmates.csv')
df_inmates.head()

,Execution No.,TDCJ Number,First Name,Last Name,Date Received,Age Received,Date of Offense,Age at Offense,Execution Date,Execution Age,...,Education Level,Weight,Eye Color,Hair Color,Native State,Last Statement,Sentiment,More Info URL,Last Statement URL,Poverty Rate
0,545,999275,Ruben,Cardenas,07/29/1998,28.0,02/22/1997,26.0,11/09/2017,47,...,11.0,204.0,Brown,Black,Mexico,This is my statement:Â My final words.Â Firs...,-1,https://www.tdcj.state.tx.us/death_row/dr_info...,https://www.tdcj.state.tx.us/death_row/dr_info...,45.4
1,544,999411,Robert,Pruett,04/30/2002,22.0,12/17/1999,20.0,10/12/2017,38,...,8.0,160.0,Brown,Blonde,Texas,I just want to let everyone in here to know I ...,-1,https://www.tdcj.state.tx.us/death_row/dr_info...,https://www.tdcj.state.tx.us/death_row/dr_info...,26.6
2,543,999494,Taichin,Preyor,03/30/2005,34.0,02/26/2004,33.0,7/27/2017,46,...,10.0,219.0,Brown,Black,Texas,"First and foremost I'd like to say, ""Justice h...",1,https://www.tdcj.state.tx.us/death_row/dr_info...,https://www.tdcj.state.tx.us/death_row/dr_info...,21.6
3,542,997,James,Bigby,NaN,NaN,NaN,NaN,3/14/2017,61,...,NaN,NaN,NaN,NaN,NaN,"Yes, I do, Grace Kehler is that you? I have gi...",-1,https://www.tdcj.state.tx.us/death_row/dr_info...,https://www.tdcj.state.tx.us/death_row/dr_info...,12.3
4,541,999145,Rolando,Ruiz,NaN,NaN,NaN,NaN,3/07/2017,44,...,NaN,NaN,NaN,NaN,NaN,"Yes sir, I would first like to say to the Sanc...",1,https://www.tdcj.state.tx.us/death_row/dr_info...,https://www.tdcj.state.tx.us/death_row/dr_info...,19.4


In [7]:
df_inmates['Age Received'].mean()

28.564935064935064

In [8]:
df_inmates['Age at Offense'].mean()

26.954545454545453

Most inmates on average are processed into death row approximately 2.5 years after their offense (quick turn around time).

In [9]:
df_inmates['Execution Age'].mean()

39.3394495412844

Most inmates on average stay on death row for approximately 11 years before being executed.

In [15]:
df_inmates['Gender'].value_counts()

Male      150
Female      3
male        1
Name: Gender, dtype: int64

Clearly, males outnumber females.